In [ ]:
knitr::opts_chunk$set(echo = TRUE)

## Target
The goal of this competition is to predict rainfall for each day of the year.

ML tools: 
  tidymodels related worksflows & glm, lightgbm, ranger engine.
Evaluatioin metric:
  roc_auc, percent of rain probability.
  sample_submission.csv : 0.5. it is value is 0. I tried change it to 1, the score is 0.5 as well.
  kaggle best score is 0.87 in the begining.
  
Notice:
- from small data, data usage is as important as feature engineering. cv with repeate could slightly reduce the variance(std_err). as side effect, the metric might be improve a bit as well.
- in last stage, consider change cv =10, repeate = 5. it is computation expensive task. only do at final stage.

## librar y & load_data

### library


In [ ]:
library(tidyverse)
library(tidymodels)
library(finetune)
library(future)
library(purrr)
library(furrr)
library(textrecipes)
library(themis)


library(bonsai)
library(lightgbm)
library(xgboost)
library(ranger)

library(readr)
library(janitor)
library(lubridate)


In [ ]:
#TODO, column 18,7 has some strange data leave it handle future. 71 lines in train, 159 line in test


### loading data



In [ ]:
data_path <- '../input/playground-series-s5e3/'
train<- 
  readr::read_csv(file.path(data_path, 'train.csv'),
                  show_col_types = FALSE)|>
  mutate(rainfall=as.factor(rainfall))
test <- 
  readr::read_csv(file.path(data_path, 'test.csv'),
                  show_col_types = FALSE)
submission <-  readr::read_csv(file.path(data_path, 'sample_submission.csv'),show_col_types = FALSE)


### quick skim



### check if train & test is same distribution



In [ ]:
get_df_var<-function(df){
  df|>
    select(-any_of(c('id','rainfall')))|>
    summarize_all(var)|>
    pivot_longer(cols=everything(),
                 names_to='feature',
                 values_to='variance')

}
list(train=train, test=test)|>
  map_dfr(\(x) get_df_var(x), .id = "dataset") |>
  pivot_wider(names_from=dataset, values_from = variance)|>
  mutate(pct_change=(train-test)/train)#|>arrange(desc(abs(diff)))


### Finding of different distribution
there is no big change found 

## EDA

### time series EDA


#### plot ts


## coding

### 1. Data Loading and Initial Exploration ----

### 2. Feature Engineering ----

-   leave it in the preprocessing recipe

### 3. Data Splitting ----

#### augment_df

#### split/cv


In [ ]:
set.seed(1234)
#train <- ts_df |> as_tibble() |>filter(source=='train')|>select(-source)
# df_split <- initial_time_split(train, prop = 0.8)
df_split <- initial_split(train, prop = 0.8, strata = rainfall)
train_set <- training(df_split)
test_set <- testing(df_split)
cv_folds <- vfold_cv(train_set,
                     v = 10
                     repeats = 5,
                     repeats = 1,
                     strata = rainfall)
#cv_folds <- train_set |> sliding_period(index=date, period='year')


### 4. Preprocessing Recipe ----

#### 4.0 v0 base_line - 


In [ ]:
rcp_bs_v0 <-
  recipe(rainfall ~ ., data = train_set) |>
  update_role(id, new_role='ID')|>
  #step_rm(date,year_offset)|>
  step_impute_median(all_numeric_predictors())|> 
  step_log(all_numeric_predictors(),offset = 1, skip = FALSE) |>
  # process the logical feature to factor feature
  step_bin2factor(all_logical_predictors())|>
  # proceeding the convert the character feature to factorical feature
  step_novel(all_nominal_predictors())|>
  step_unknown(all_nominal_predictors()) |>
  step_other(all_nominal_predictors())|>
  step_dummy(all_nominal_predictors(),one_hot = TRUE) |>
  step_nzv(all_predictors())|>
  #step_corr(all_numeric_predictors())|>
  step_normalize(all_numeric_predictors())|> # Scale numeric predictors
  #step_smote(rainfall,over_ratio = 1,skip = TRUE)|>
  check_missing(all_predictors())



#### 4.0 v0_wd_smote


In [ ]:
rcp_bs_v0_wd_smote <-
  recipe(rainfall ~ ., data = train_set) |>
  update_role(id, new_role='ID')|>
  #step_rm(date,year_offset)|>
  step_impute_linear(winddirection)|>
  #step_impute_median(all_numeric_predictors())|> 
  step_log(all_numeric_predictors(),offset = 1, skip = FALSE) |>
  # process the logical feature to factor feature
  step_bin2factor(all_logical_predictors())|>
  # proceeding the convert the character feature to factorical feature
  step_novel(all_nominal_predictors())|>
  step_unknown(all_nominal_predictors()) |>
  step_other(all_nominal_predictors())|>
  step_dummy(all_nominal_predictors(),one_hot = TRUE) |>
  step_nzv(all_predictors())|>
  #step_corr(all_numeric_predictors())|>
  step_normalize(all_numeric_predictors())|> # Scale numeric predictors
  step_smote(rainfall,over_ratio = 1,skip = TRUE)|>
  check_missing(all_predictors())



#### 4.1 v1 base_line - bad


In [ ]:
rcp_bs_v1 <-
  recipe(rainfall ~ ., data = train_set) |>
  update_role(id, new_role='ID')|>
  #step_date(date, features=c('month','week'))|>
   step_mutate(
   temp_range= maxtemp - mintemp,
   heat_index= temparature+ 0.5 * (temparature- 10) * (humidity / 100),
   dew_dep= temparature - dewpoint,
   cloud_sun_ratio = cloud / sunshine + 1,    
   pressure_change = c(NA, diff(pressure)),
   air_density = 1.225 , 
   wind_power = 0.5 * air_density * windspeed**3,
   wind_direction_cat = cut(winddirection, 
                            breaks=c(0, 90, 180, 270, 360), 
                            labels=c("North", "East", "South", "West"), 
                            include.lowest=TRUE),
   cloud_humidity = cloud + humidity,
   cloud_humidity_sunshine = cloud + humidity + sunshine,
   cloud_x_sunshine = cloud * sunshine,
   humidity_x_sunshine = humidity * sunshine)|>
  step_rm(date,year_offset)|>
  step_impute_median(all_numeric_predictors())|> 
  step_log(all_numeric_predictors(),offset = 1, skip = FALSE) |>
  step_lag(all_numeric_predictors(),lag =c(1))|>
  step_impute_median(all_numeric_predictors())|> 
  # process the logical feature to factor feature
  step_bin2factor(all_logical_predictors())|>
  # proceeding the convert the character feature to factorical feature
  step_novel(all_nominal_predictors())|>
  step_unknown(all_nominal_predictors()) |>
  step_other(all_nominal_predictors())|>
  step_dummy(all_nominal_predictors(),one_hot = TRUE) |>
  step_nzv(all_predictors())|>
  step_corr(all_numeric_predictors())|>
  step_normalize(all_numeric_predictors())|> # Scale numeric predictors

  check_missing(all_predictors())



#### 4.2 v2 base_line - 


In [ ]:
# df["Temp_Range"] = df["maxtemp"] - df["mintemp"]
#     df["Heat_Index"] = df["temparature"] + 0.5 * (df["temparature"] - 10) * (df["humidity"] / 100)    
#     df["Dew_Dep"] = df["temparature"] - df["dewpoint"]    
#     df["Wind_Chill"] = np.where(
#         df["temparature"] < 10,
#         13.12 + 0.6215 * df["temparature"] - 11.37 * df["windspeed"]**0.16 + 
#        0.3965 * df["temparature"] * df["windspeed"]**0.16,
#         df["temparature"]
#     )    
#     df["Cloud_Sun_Ratio"] = df["cloud"] / (df["sunshine"] + 1)    
#     df["Pressure_Change"] = df["pressure"].diff().fillna(0)    
#     air_density = 1.225  
#     df["Wind_Power"] = 0.5 * air_density * df["windspeed"]**3         
#     df["Wind_Direction_Cat"] = pd.cut(df["winddirection"], 
#                                       bins=[0, 90, 180, 270, 360], 
#                                       labels=["North", "East", "South", "West"], 
#                                       include_lowest=True)
# 
#     df['cloud + humidity'] = df.cloud + df.humidity
#     df['cloud + humidity + sunshine'] = df.cloud + df.humidity + df.sunshine
#     df['cloud * sunshine'] = df.cloud * df.sunshine
#     df['humidity * sunshine'] = df.humidity * df.sunshine
# 
#     df["HTI"] = df["temparature"] + 0.2 * df["humidity"]    
#     df["ACI"] = (df["maxtemp"] + df["mintemp"]) / 2 - (df["humidity"] / 2) + df["dewpoint"]    
#     df["CSI"] = (df["sunshine"] - df["cloud"]) / (df["sunshine"] + df["cloud"] + 1)  
#     df["WCI"] = df["temparature"] - 0.5 * df["windspeed"]
#     df['day'] = pd.to_datetime(df['day'], errors='coerce')    
#    
#     df['month'] = df['day'].dt.month
#     df['day_of_week'] = df['day'].dt.dayofweek
#     df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)    
#     df['temp_range'] = df['maxtemp'] - df['mintemp']
#     df['avg_temp'] = (df['maxtemp'] + df['mintemp']) / 2
#     df['temp_deviation'] = df['temparature'] - df['avg_temp']   
#   
#     df['dew_point_depression'] = df['temparature'] - df['dewpoint']    
#    
#     df['wind_dir_rad'] = np.deg2rad(df['winddirection'])
#     df['wind_dir_sin'] = np.sin(df['wind_dir_rad'])
#     df['wind_dir_cos'] = np.cos(df['wind_dir_rad'])
#     df.drop(columns=['wind_dir_rad'], inplace=True)   
#     df['wind_chill'] = 13.12 + 0.6215 * df['temparature'] - 11.37 * (df['windspeed']**0.16) + 0.3965 * df['temparature'] * (df['windspeed']**0.16)
#     df['humidity_temp'] = df['humidity'] * df['temparature']
#     df['cloud_sunshine'] = df['cloud'] * df['sunshine']
#     df['rolling_temp_mean'] = df['avg_temp'].rolling(window=7).mean()
#     df['rolling_wind_mean'] = df['windspeed'].rolling(window=7).mean()
#     df['rolling_humidity_mean'] = df['humidity'].rolling(window=7).mean()
#     df['temp_lag_1'] = df['avg_temp'].shift(1)
#     df['humidity_lag_1'] = df['humidity'].shift(1)
#     df['windspeed_lag_1'] = df['windspeed'].shift(1)
#     df['pressure_temp_interaction'] = df['pressure'] * df['avg_temp']
#     df['windspeed_temp_interaction'] = df['windspeed'] * df['avg_temp']
#     df['sunshine_cloud_interaction'] = df['sunshine'] * df['cloud']
#     df['season'] = df['month'].apply(lambda x: 'Spring' if 3 <= x <= 5 else
#                                       'Summer' if 6 <= x <= 8 else
#                                       'Autumn' if 9 <= x <= 11 else 'Winter')
#     for c in ['pressure', 'maxtemp', 'temparature', 'humidity']:
#         for gap in [1]:
#             df[c+f"_shift{gap}"] = df[c].shift(gap)
#             df[c+f"_diff{gap}"] = df[c].diff(gap)
#     df = pd.get_dummies(df, columns=['season'], drop_first=True)  
#     df.drop(columns=['day'], inplace=True


In [ ]:
rcp_bs_v2 <-
  recipe(rainfall ~ ., data = train_set) |>
  update_role(id, new_role='ID')|>
  step_rm(date,year_offset)|>
  step_impute_median(all_numeric_predictors())|> 
  step_mutate(
   temp_range= maxtemp - mintemp,
   heat_index= temparature+ 0.5 * (temparature- 10) * (humidity / 100),
   dew_dep= temparature - dewpoint,
   cloud_sun_ratio = cloud / sunshine + 1,    
   pressure_change = c(NA,diff(pressure)),
   air_density = 1.225 , 
   wind_power = 0.5 * air_density * windspeed**3,
   wind_direction_cat = cut(winddirection, 
                            breaks=c(0, 90, 180, 270, 360), 
                            labels=c("North", "East", "South", "West"), 
                            include.lowest=True),
   cloud_humidity = cloud + humidity,
   cloud_humidity_sunshine = cloud + humidity + sunshine,
   cloud_x_sunshine = cloud * sunshine,
   humidity_x_sunshine = humidity * sunshine)|>
  step_impute_median(all_numeric_predictors())|> 
  step_log(all_numeric_predictors(),offset = 1, skip = FALSE) |>
  # process the logical feature to factor feature
  step_bin2factor(all_logical_predictors())|>
  # proceeding the convert the character feature to factorical feature
  step_novel(all_nominal_predictors())|>
  step_unknown(all_nominal_predictors()) |>
  step_other(all_nominal_predictors())|>
  step_dummy(all_nominal_predictors(),one_hot = TRUE) |>
  step_nzv(all_predictors())|>
  #step_corr(all_numeric_predictors())|>
  step_normalize(all_numeric_predictors())|> # Scale numeric predictors
  #step_smote(rainfall,over_ratio = 1,skip = TRUE)|>
  check_missing(all_predictors())


#### 4.10 all recipes



In [ ]:
set.seed(1234)
library(future)
library(furrr)
selected_rcps <- list(#bs=rcp_bs_v0
                      bs_wd_smote=rcp_bs_v0_wd_smote
                      #bs_v1=rcp_bs_v1#, # is significanttly change the data , and performance is descress . give it up.
                      #bs_v2=rcp_bs_v2
                      )
#plan(multisession,workers = 5)
#selected_rcps|>map(\(rcp_item) rcp_item|>prep()|>bake(new_data=train_set)|>summary())
#plan(sequential)


### 5. Model Specification ----



In [ ]:
glm_eng <- 
  logistic_reg(penalty = 0.01623777,
               mixture = 0.05) |>  # Example penalty and mixture values
  set_engine("glmnet") |>
  set_mode("classification")    # Specify classification

lgbm_eng<-
   parsnip::boost_tree(
      trees = 500, # Number of trees
      #learn_rate = 0.01,
      tree_depth =5,
      loss_reduction = 0.001,
      stop_iter = 50,  
      #sample_size = 0.9, # Added sample_size
      #tree_depth = tune(),
      #mtry = 0.5,
      min_n = 200
   ) |>
   set_mode("classification")|>
   set_engine("lightgbm",
              #metric='roc_auc', 
              num_leaves = 20,
              counts = FALSE,
              metric = "auc",              # 优化目标
              verbose=1) 

rf_eng<- rand_forest( trees = 700, 
                      #learn_rate = 0.01,
                      #sample_size=0.8,
                      #mtry=10, 
                      min_n=200) |>
  set_engine("ranger",num.threads=4)|>
  set_mode("classification") 

xgb_eng<- parsnip::boost_tree( trees = 500, 
                      learn_rate = 0.01,
                      loss_reduction = 0.001,
                      sample_size = 0.8, # Added sample_size
                      #mtry=tune(),
                      min_n=70) |>
  set_engine("xgboost",num.threads=8)|>
  set_mode("classification")


c50_eng <- boost_tree() |>
  set_mode('classification')|>
  set_engine('C5.0')

earth_eng <-  # good model base score 0.8718
  mars() %>% 
  set_mode("classification") %>% 
  set_engine("earth") 

svm_eng <- 
  svm_rbf(
    cost = 0.001, 
    rbf_sigma = 0.01
    ) %>% 
  set_mode("classification") 

kknn_eng <- 
  nearest_neighbor(neighbors = 5, 
                   #weight_func = tune()
                   ) %>% 
  set_mode("classification") %>% 
  set_engine("kknn") 

selected_eng <- list(glm=glm_eng,
                     rf=rf_eng,
                     #lgbm=lgbm_eng,
                     xgb=xgb_eng,
                     #c50=c50_eng,
                     earth= earth_eng,
                     #kknn=kknn_eng,
                     svm=svm_eng
                     
                     )


### 6. Workflow ----
#### set metrics


In [ ]:
rocauc_metrics <- metric_set(roc_auc) # main goal is roc_auc, accuracy is just for reference

#### simple wflow



In [ ]:
set.seed(1234)

simple_wf_fit <- 
  workflow() |>
  add_recipe(rcp_bs_v0_wd_smote) |>
  add_model(svm_eng)|>
  # fit(train_set)
  fit_resamples(cv_folds,
  #  last_fit(df_split,
          control = ctrl,
           metrics=rocauc_metrics)
#plan(sequential)
 simple_wf_fit |> collect_metrics()
#   extract_fit_engine()|>
#   plot()


#### simple workflowset



In [ ]:
set.seed(1234)
library(future)
plan(multisession,workers = 12)
ctrl <- control_resamples(save_pred = TRUE, save_workflow = TRUE,verbose=TRUE)
wfs_result <-
  workflow_set(preproc = selected_rcps,
               models = selected_eng ) |>
  workflow_map(fn='fit_resamples',
               resamples =cv_folds,
               metrics =rocauc_metrics,
               control = ctrl
               )
wfs_result|> 
  collect_metrics()  |>
  filter(.metric=='roc_auc')|>
  select(wflow_id, model, .metric,mean, n, std_err
         )
  
plan(sequential)


### 7 stacking



In [ ]:
set.seed(1234)
library(future)
#plan(multisession,workers = 4)
combined_fit <-
  stacks::stacks()|>
  stacks::add_candidates(wfs_result)|>
  stacks::blend_predictions()|>
  stacks::fit_members()

combined_fit|>
  autoplot(type = "weights")

autoplot(combined_fit)
#plan(sequential)


### 7. Tuning Grid ----
#### tune lgbm



#### tune glmnet


### 8. Cross-Validation ----



### 9. Tuning and Evaluation ----



### 10. Evaluate on Test Set ----



In [ ]:
combined_test_result <- 
  test_set %>%
  bind_cols(predict(combined_fit, 
                    new_data=test_set,type='prob'))
combined_test_result|>roc_auc(rainfall, .pred_1,event_level = 'second')


### 11. Prepare Submission ----



In [ ]:
set.seed(1234)
library(future)
plan(multisession,workers = 12)
final_model <- combined_fit#simple_wf_fit|>extract_workflow()
#final_model <- simple_wf_fit|>extract_workflow()
final_predictions <- final_model |>
   predict(new_data = test,
           type='prob') 
plan(sequential)

 # #Handle negative predictions
 # final_predictions <- final_predictions |>
 #   mutate(.pred= ifelse(.pred< 0, 0, .pred))

 # Save submission file
 submission |>
   mutate(rainfall=final_predictions$.pred_1)|>
   readr::write_csv("submission.csv")
 zip('submission.csv.zip','submission.csv')
 
